In [32]:
def locate_hafstar(df, uid=461120):
    
    return bool(df.loc[df['uid'] == uid].shape[0])

locate_hafstar(tags)

False

In [1]:
import json
import numpy as np
import pandas as pd

%matplotlib inline
# import matplotlib as plt
import matplotlib.pyplot as pyplt

import seaborn as sns

import collections
import csv
import sys

from collections import defaultdict
import networkx as nx
try:
    import simplejson as json
except:
    import json

print "imported"

imported


In [38]:
tags_dir = '../data/2016_2019/full/tags/'
tags = pd.read_csv(tags_dir + 'tags.csv')
tags.sample(5)

,nid,uid,tid,name,power_type,count,date,power
46917,12620,379,13069,lean,npt,2.0,2015,False
40600,11331,422561,8485,silica,npt,67.0,2014,False
27483,64,1,119,nrg,npt,47.0,2012,False
38866,10586,1,707,new-york,npt,19.0,2014,False
25144,13725,498199,14322,uspstracking,npt,1.0,2016,False


In [8]:
grouped = tags.groupby('nid')
co_tags = [group['name'].tolist() for nid, group in grouped]
# len(grouped), len(co_tags) # (9561, 9561)
# co_tags[0] # ['attachments', 'content', 'slider']

In [39]:
nt = 256
y = 2018

print tags_dir + 'graphml/' 'tags_' + str(nt) + '.graphml'
print tags_dir + 'graphml/by_year/' + 'tags_' + str(year) + '_' + str(num_tags) + '.graphml'

../data/2016_2019/full/tags/graphml/tags_256.graphml
../data/2016_2019/full/tags/graphml/by_year/tags_2018_512.graphml


### CODE FROM: Piotr Migdal
https://github.com/stared/tag-graph-map-of-stackexchange

### ALL TAGS

In [40]:
bun = Bundle()

for num_tags in [512, 256, 128, 64, 32]:
    
    # PROBABLY AN HAVE THIS OUTSIDE LOOP BUT GIVEN THE TIME I KNOW THIS'LL WORK!
    bun = Bundle(co_tags)
    
    bun.filter_elements(first_n=num_tags)  # takes only 100 most frequent tags
    bun.calculate_pair_weights(func=oe_ratio, threshold=1.0)
    bun.export2graphml(tags_dir + 'graphml/' 'tags_' + str(num_tags) + '.graphml')

Saved!
Nodes: 512	Edges: 5515
Saved!
Nodes: 256	Edges: 2726
Saved!
Nodes: 128	Edges: 1204
Saved!
Nodes: 64	Edges: 475
Saved!
Nodes: 32	Edges: 155


### TAGS BY YEAR

In [45]:
# PROBABLY CAN DO THIS WITH FILTERING THE ONE ABOVE, BUT AGAIN, PRESSED FOR TIME
# tags_2012.csv tags_2013.csv tags_2014.csv tags_2015.csv tags_2016.csv tags_2017.csv tags_2018.csv

by_year_dir = tags_dir + 'graphml/by_year/'
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018]

for year in years:

    tag_year = tags.loc[tags['date'] == year]
    grouped = tag_year.groupby('nid')
    co_tags = [group['name'].tolist() for nid, group in grouped]

    bun = Bundle()
    
    print year
    
    for num_tags in [512, 256, 128, 64, 32]:
        
        out_dir = tags_dir + 'graphml/by_year/' 
        tag_file = 'tags_' + str(year) + '_' + str(num_tags) + '.graphml'

        # PROBABLY AN HAVE THIS OUTSIDE LOOP BUT GIVEN THE TIME I KNOW THIS'LL WORK!
        bun = Bundle(co_tags)

        bun.filter_elements(first_n=num_tags)  # takes only 100 most frequent tags
        bun.calculate_pair_weights(func=oe_ratio, threshold=1.0)
        bun.export2graphml(by_year_dir + tag_file)
    print ''

2012
Saved! Nodes: 512	Edges: 1870
Saved! Nodes: 256	Edges: 1172
Saved! Nodes: 128	Edges: 572
Saved! Nodes: 64	Edges: 217
Saved! Nodes: 32	Edges: 79

2013
Saved! Nodes: 512	Edges: 3325
Saved! Nodes: 256	Edges: 1776
Saved! Nodes: 128	Edges: 847
Saved! Nodes: 64	Edges: 357
Saved! Nodes: 32	Edges: 100

2014
Saved! Nodes: 512	Edges: 4377
Saved! Nodes: 256	Edges: 2354
Saved! Nodes: 128	Edges: 1056
Saved! Nodes: 64	Edges: 416
Saved! Nodes: 32	Edges: 112

2015
Saved! Nodes: 512	Edges: 3323
Saved! Nodes: 256	Edges: 1754
Saved! Nodes: 128	Edges: 867
Saved! Nodes: 64	Edges: 364
Saved! Nodes: 32	Edges: 105

2016
Saved! Nodes: 512	Edges: 4413
Saved! Nodes: 256	Edges: 2394
Saved! Nodes: 128	Edges: 1061
Saved! Nodes: 64	Edges: 357
Saved! Nodes: 32	Edges: 92

2017
Saved! Nodes: 512	Edges: 6258
Saved! Nodes: 256	Edges: 3265
Saved! Nodes: 128	Edges: 1326
Saved! Nodes: 64	Edges: 514
Saved! Nodes: 32	Edges: 152

2018
Saved! Nodes: 512	Edges: 5515
Saved! Nodes: 256	Edges: 2726
Saved! Nodes: 128	Edges: 120

In [42]:
def oecsv2graphml(input_path, output_path="output.graphml",
                  threshold=0.):
    with open(input_path, "r") as f:
        data = list(csv.reader(f))[1:]
    data = map(lambda x: [x[0], x[1], float(x[2]), int(x[3]), int(x[4]), int(x[5])], data)
    nodes = dict([(x[0], x[3]) for x in data]
                 + [(x[1], x[4]) for x in data])

    G = nx.Graph()
    for k, v in nodes.items():
        G.add_node(k, weight=v)

    data_filtered = filter(lambda x: x[2] >= threshold, data)
    for x in data_filtered:
        G.add_edge(x[0], x[1], weight=x[2])

    nx.write_graphml(G, output_path)
    print "Saved! Nodes: %d\tEdges: %d out of %d" % (len(nodes), len(data_filtered), len(data))


# if __name__ == "__main__":
#     oecsv2graphml(*sys.argv[1:])

In [41]:
def oe_ratio(n, nx, ny, nxy):
    return float(n) * float(nxy) / (float(nx) * float(ny))

def cooccurrences(n, nx, ny, nxy):
    return float(nxy)


class Bundle():
    def __init__(self, li=[], json_path=""):
        if json_path:
            with open(json_path, "r") as f:
                li = json.load(f)
        self.total = len(li)
        self.elements_dict_full = self.bundles2tags(li)
        self.pairs_dict_full = self.bundles2pairs(li)
        self.filter_elements()

    def filter_elements(self, min_counts=0, first_n=0):
        self.elements = self.elements_dict_full.items()
        self.elements.sort(key=lambda x: -x[1])

        if first_n:
            self.elements = self.elements[:first_n]
        if min_counts:  # if not necessary, but can make it faster
            self.elements = filter(lambda x: x[1] >= min_counts, self.elements)

        elements_dict = dict(self.elements)  # just to make "in" quicker

        self.pairs = self.pairs_dict_full.items()
        self.pairs = filter(lambda x: x[0][0] in elements_dict
                            and x[0][1] in elements_dict,
                            self.pairs)
        self.pairs.sort(key=lambda x: -x[1])
        return {"elements": len(self.elements), "pairs": len(self.pairs)}

    def calculate_pair_weights(self, func=oe_ratio, threshold=0.):
        elements_dict = dict(self.elements)
        self.pairs_weighted = [(xy, func(n=self.total, nx=elements_dict[xy[0]],
                                ny=elements_dict[xy[1]], nxy=nxy))
                               for xy, nxy in self.pairs]
        self.pairs_weighted = filter(lambda x: x[1] >= threshold,
                                     self.pairs_weighted)
        self.pairs_weighted.sort(key=lambda x: -x[1])
        return len(self.pairs_weighted)

    def bundles2tags(self, li):
        """For a list with elements being lists, counts all elements inside,
           e.g. [[1, 6, 2], [3], [3, 1, 2]] -> defaultdict{1:2, 2:2, 3:2, 6:1}"""
        res = defaultdict(lambda: 0)
        for bundle in li:
            for x in bundle:
                res[x] += 1
        return res

    def bundles2pairs(self, li):
        """For a list with elements being lists, counts all lexically ordered pairs,
           e.g. [[1, 6, 2], [3], [3, 1, 2]] -> defaultdict{(1,2):2, (1,3):1, (1,6):1}"""
        res = defaultdict(lambda: 0)
        for bundle in li:
            for x in bundle:
                for y in bundle:
                    if x < y:
                        res[(x, y)] += 1
        return res

    def export2graphml(self, output_path):  # to do
        G = nx.Graph()
        for k, v in self.elements:
            G.add_node(k, weight=v)

        for x in self.pairs_weighted:
            G.add_edge(x[0][0], x[0][1], weight=x[1])

        nx.write_graphml(G, output_path)
        print "Saved! Nodes: %d\tEdges: %d" % (len(self.elements), len(self.pairs_weighted))


In [ ]:
2012
Nodes: 512	Edges: 1870
Nodes: 256	Edges: 1172
Nodes: 128	Edges: 572
Nodes: 64	Edges: 217
Nodes: 32	Edges: 79

2013
Nodes: 512	Edges: 3325
Nodes: 256	Edges: 1776
Nodes: 128	Edges: 847
Nodes: 64	Edges: 357
Nodes: 32	Edges: 100

2014
Nodes: 512	Edges: 4377
Nodes: 256	Edges: 2354
Nodes: 128	Edges: 1056
Nodes: 64	Edges: 416
Nodes: 32	Edges: 112

2015
Nodes: 512	Edges: 12563
Nodes: 256	Edges: 7517
Nodes: 128	Edges: 5385
Nodes: 64	Edges: 1420
Nodes: 32	Edges: 268
        
2016
Nodes: 512	Edges: 11434
Nodes: 256	Edges: 7466
Nodes: 128	Edges: 2493
Nodes: 64	Edges: 333
Nodes: 32	Edges: 92
        
2027    
Nodes: 512	Edges: 6258
Nodes: 256	Edges: 3265
Nodes: 128	Edges: 1326
Nodes: 64	Edges: 514
Nodes: 32	Edges: 152
        
2018
Nodes: 512	Edges: 5515
Nodes: 256	Edges: 2726
Nodes: 128	Edges: 1204
Nodes: 64	Edges: 475
Nodes: 32	Edges: 155